SK TIME REGRESSION 

In [644]:
#   IMPORTS
from sktime.utils.plotting import plot_series
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
from sktime.performance_metrics.forecasting import mean_absolute_error
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.fbprophet import Prophet
from sktime.forecasting.ets import AutoETS
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.bats import BATS
from sktime.forecasting.tbats import TBATS
from sktime.forecasting.var import VAR
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.sarimax import SARIMAX
from sktime.forecasting.theta import ThetaForecaster

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from prophet.forecaster import Prophet 

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

from pandas import read_csv
import pandas as pd 
import numpy as np
import warnings
import datetime
import warnings
warnings.filterwarnings("ignore")


In [645]:
#   LOAD IN DATA
data = pd.read_csv('data_for_inequalities.csv')
#View head
data.head()

,Date,SG_Y
0,2017/12,76.500
1,2018/01,82.750
2,2018/02,85.500
3,2018/03,86.125
4,2018/04,88.700


In [646]:
#   USER DEFINED
#Set target_col to column name you want to forecast 
#Example uses column named SG_Y
target_col = 'SG_Y'

In [647]:
#   TRAIN/TEST SPLIT 
y = data[target_col]
y_train, y_test = temporal_train_test_split(y, test_size=12)
y_test

36    104.250000
37    103.458333
38    103.916667
39    108.541667
40    108.458333
41    107.291667
42    106.583333
43    101.208333
44    100.125000
45     99.791667
46    100.791667
47    102.958333
Name: SG_Y, dtype: float64

In [648]:
#   Set Date
data['Date'][36]

'2021/12'

In [649]:
#Set Forecasting horizon ------ 12 months
fh = np.arange(1, len(y_test)+1)
fh

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

UNIVARIATRE REGRESSION MODELS 

In [650]:

#   Build Random Forest Model and score MAE in sample Forecast
r_forest = RandomForestRegressor(random_state = 0)
RFforecaster = make_reduction(r_forest, window_length = 12)
#Fits model on training data
RFforecaster.fit(y_train)
#Forecasts
RFy_preds = RFforecaster.predict(fh)
#Calculates MAE for Random Forest model
RFmae = mean_absolute_error(y_test, RFy_preds)
#Prints MAE score for model
print('Random Forest Model - MAE is %.6f' % (RFmae))
print('--------Random Forest Forecast--------')
RFy_preds

Random Forest Model - MAE is 2.606042
--------Random Forest Forecast--------


36    104.089167
37    104.138333
38    104.135000
39    104.172500
40    104.088333
41    104.195000
42    104.084167
43    104.195000
44    104.084167
45    104.195000
46    104.084167
47    104.195000
dtype: float64

In [651]:
#   Build Decision Tree Model and score MAE in sample Forecast
d_tree = DecisionTreeRegressor(random_state = 0)
DTforecaster = make_reduction(d_tree, window_length = 12)
#Fits model on training data
DTforecaster.fit(y_train)
#Forecasts
DTy_preds = DTforecaster.predict(fh)
#Calculates MAE for Decision Tree model
DTmae = mean_absolute_error(y_test, DTy_preds)
#Prints MAE score for model
print('Decision Tree Model - MAE is %.6f' % (DTmae))
print('--------Decision Tree Forecast--------')
DTy_preds

Decision Tree Model - MAE is 2.579861
--------Decision Tree Forecast--------


36    104.333333
37    103.916667
38    104.416667
39    103.916667
40    104.416667
41    103.916667
42    104.416667
43    103.916667
44    104.416667
45    103.916667
46    104.416667
47    103.916667
dtype: float64

In [652]:
#   Build XGBoost Model and score MAE in sample Forecast
xgb = XGBRegressor(random_state = 0, objective='reg:squarederror', n_estimators = 1000)
XGBforecaster = make_reduction(xgb, window_length = 12, strategy="recursive")
#Fits model on training data
XGBforecaster.fit(y_train)
#Forecasts
XGBy_preds = XGBforecaster.predict(fh)
#Calculates MAE for XGBoost
XGBmae = mean_absolute_error(y_test, XGBy_preds)
#Prints MAE score for model
print('XGBOOST Model - MAE is %.6f' % (XGBmae))
print('--------XGBoost Forecast--------')
XGBy_preds

XGBOOST Model - MAE is 2.593700
--------XGBoost Forecast--------


36    103.999245
37    104.329330
38    103.836288
39    103.922264
40    103.922264
41    103.922264
42    103.922264
43    103.922264
44    103.922264
45    103.922264
46    103.922264
47    103.922264
dtype: float64

In [653]:
#   Build K Nearest Neighbors Model and score MAE in sample Forecast
kNN = KNeighborsRegressor(n_neighbors=1)
KNNforecaster = make_reduction(kNN, window_length=12, strategy="recursive")
#Fits model on training data
KNNforecaster.fit(y_train)
#Forecasts
KNNy_preds = KNNforecaster.predict(fh)
#Calculates MAE for KNN
KNNmae = mean_absolute_error(y_test, KNNy_preds)
#Prints MAE score for model
print('K-Nearest Neighbors Model - MAE is %.6f' % (KNNmae))
print('--------K-NN Forecast--------')
KNNy_preds

K-Nearest Neighbors Model - MAE is 2.559028
--------K-NN Forecast--------


36    103.916667
37    103.916667
38    103.916667
39    103.916667
40    103.916667
41    103.916667
42    103.916667
43    103.916667
44    103.916667
45    103.916667
46    103.916667
47    103.916667
dtype: float64

In [654]:
#   Build AUTO ARIMA Model and score MAE in sample Forecast
AAforecaster = AutoARIMA(sp=12)
#Fits model on training data
AAforecaster.fit(y_train)
#Forecasts
AAy_preds = AAforecaster.predict(fh)
#Calculates MAE for Auto ARIMA
autoArimaMAE = mean_absolute_error(y_test, AAy_preds)
#Prints MAE score for model
print('Auto ARIMA Model - MAE is %.6f' % (autoArimaMAE))
print('--------Auto ARIMA Forecast--------')
AAy_preds

Auto ARIMA Model - MAE is 6.104247
--------Auto ARIMA Forecast--------


36    104.606131
37    106.023768
38    107.734597
39    109.315119
40    109.625139
41    110.065467
42    111.059604
43    110.099123
44    111.158413
45    112.543473
46    113.505033
47    114.890093
dtype: float64

In [655]:
#   Build ARIMA Model and score MAE in sample Forecast
Aforecaster = ARIMA(order=(1,0,1))

#Fits model on training data
Aforecaster.fit(y_train)
#Forecasts
Ay_preds = Aforecaster.predict(-fh)
#Calculates MAE for Auto ARIMA
ArimaMAE = mean_absolute_error(y_test, Ay_preds)
#Prints MAE score for model
print('ARIMA Model - MAE is %.6f' % (ArimaMAE))
print('--------ARIMA Forecast--------')
Ay_preds

ARIMA Model - MAE is 7.521689
--------ARIMA Forecast--------


23     92.321507
24     97.198792
25     96.302627
26     95.893231
27     94.143612
28     93.547725
29     97.139154
30     98.163094
31     98.386985
32    106.331560
33    102.874812
34    103.858961
dtype: float64

In [656]:
#   Build SARIMAX Model and score MAE in sample Forecast
Sforecaster = SARIMAX(trend='t', seasonal_order=(1, 0, 0, 12))
#Fits model on training data
Sforecaster.fit(y_train)
#Forecasts
Sy_preds = Sforecaster.predict(fh)
#Calculates MAE for SARIMAX
SarimaxMAE = mean_absolute_error(y_test, Sy_preds)
#Prints MAE score for model
print('SARIMAX Model - MAE is %.6f' % (SarimaxMAE))
print('--------SARIMAX Forecast--------')
Sy_preds

SARIMAX Model - MAE is 9.939516
--------SARIMAX Forecast--------


36    105.138474
37    106.942127
38    109.038337
39    111.068942
40    112.069702
41    113.225204
42    114.893564
43    114.953479
44    116.765914
45    118.898294
46    120.716968
47    122.938188
Name: SG_Y, dtype: float64

In [657]:

from sktime.forecasting.fbprophet import Prophet
#   Build Prophet Model and Score MAE in sample Forecast
Pforecaster = Prophet()
#Fits model on training data
Pforecaster.fit(y_train)
#Forecasts
PROPHy_preds = Pforecaster.predict(fh)
#Calculates MAE for KNN
Proph_mae = mean_absolute_error(y_test, PROPHy_preds)
#Prints MAE score for model
print('PROPHET Model - MAE is %.6f' % (Proph_mae))
print('--------Prophet Forecast--------')
PROPHy_preds

14:35:54 - cmdstanpy - INFO - Chain [1] start processing
14:35:55 - cmdstanpy - INFO - Chain [1] done processing


PROPHET Model - MAE is 6.942886
--------Prophet Forecast--------


36    105.038421
37    106.195435
38    108.219995
39    109.443236
40    109.607162
41    110.253746
42    110.377098
43    111.648478
44    112.805492
45    114.830051
46    116.053293
47    116.217219
dtype: float64

In [658]:
#   Build Auto ETS Model and score MAE in sample Forecast
ETSforecaster = AutoETS(auto=True, sp=4, n_jobs=-1)
#Fits model on training data
ETSforecaster.fit(y_train)
#Forecasts
ETSy_preds = ETSforecaster.predict(fh)
#Calculates MAE for ETS 
ETSmae = mean_absolute_error(y_test, ETSy_preds, symmetric=False)
#Prints MAE score for model
print('AUTO ETS Model - MAE is %.6f' % (ETSmae))
print('--------Auto ETS Forecast--------')
ETSy_preds

AUTO ETS Model - MAE is 2.600905
--------Auto ETS Forecast--------


36    103.960567
37    103.995618
38    104.023659
39    104.046092
40    104.064038
41    104.078394
42    104.089880
43    104.099068
44    104.106419
45    104.112300
46    104.117004
47    104.120768
dtype: float64

In [659]:
#   Build Exponential Smoothing Model and score MAE in sample Forecast
ESforecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=12)
#Fits model on training data
ESforecaster.fit(y_train)
#Forecasts
ESy_preds = ESforecaster.predict(fh)
#Calculates MAE for Exponential Smoothing
ESmae = mean_absolute_error(y_test, ESy_preds, symmetric=False)
#Prints MAE score for model
print('Exponential Smoothing Model - MAE is %.6f' % (ESmae))
print('--------Exponential Smoothing Forecast--------')
ESy_preds

Exponential Smoothing Model - MAE is 5.073832
--------Exponential Smoothing Forecast--------


36    103.370652
37    103.458847
38    102.519828
39    100.345130
40     99.543366
41     99.094908
42     98.062865
43     98.437097
44     96.397269
45     95.312068
46     94.812829
47     95.135187
Name: SG_Y, dtype: float64

In [660]:
#   Build BATS Model and score MAE in sample Forecast
BATSforecaster = BATS(sp=12, use_trend=True, use_box_cox=False)
#Fits model on training data
BATSforecaster.fit(y_train)
#Forecasts
BATSy_preds = BATSforecaster.predict(fh)
#Calculates MAE for BATS Model
BATSmae = mean_absolute_error(y_test, BATSy_preds)
#Prints MAE score for model
print('BATS Model - MAE is %.6f' % (BATSmae))
print('--------BATS Model Forecast--------')
BATSy_preds

BATS Model - MAE is 3.050711
--------BATS Model Forecast--------


36    104.148186
37    104.329425
38    104.510663
39    104.691902
40    104.873140
41    105.054379
42    105.235617
43    105.416856
44    105.598094
45    105.779333
46    105.960571
47    106.141810
Name: SG_Y, dtype: float64

In [661]:
#   Build TBATS Model and score MAE in sample Forecast
TBATSforecaster = TBATS(sp=12, use_trend=True, use_box_cox=False)
#Fits model on training data
TBATSforecaster.fit(y_train)
#Forecasts
TBATSy_preds = TBATSforecaster.predict(fh)
#Calculates MAE for BATS Model
TBATSmae = mean_absolute_error(y_test, TBATSy_preds)
#Prints MAE score for model
print('TBATS Model - MAE is %.6f' % (TBATSmae))
print('--------TBATS Model Forecast--------')
TBATSy_preds

TBATS Model - MAE is 3.050711
--------TBATS Model Forecast--------


36    104.148186
37    104.329425
38    104.510663
39    104.691902
40    104.873140
41    105.054379
42    105.235617
43    105.416856
44    105.598094
45    105.779333
46    105.960571
47    106.141810
Name: SG_Y, dtype: float64

In [662]:
#Build Naive Forecaster Model and score MAE in sample Forecast
Naiveforecaster = NaiveForecaster(strategy="last", sp=12)
#Fits Model on training data
Naiveforecaster.fit(y_train)
#Forecasts
Naive_y_preds = Naiveforecaster.predict(fh)
#Calculates MAE for BATS Model
NaiveMAE = mean_absolute_error(y_test, Naive_y_preds)
#Prints MAE score for model
print('Naive Forecaster Model - MAE is %.6f' % (NaiveMAE))
print('--------Naive Model Forecast--------')
Naive_y_preds

Naive Forecaster Model - MAE is 6.996528
--------Naive Model Forecast--------


36     96.750000
37     96.166667
38     94.833333
39     93.833333
40     96.083333
41     98.000000
42     98.500000
43    104.000000
44    104.333333
45    103.833333
46    104.416667
47    103.916667
dtype: float64

In [663]:
#   Build LGBM Model and score MAE in sample forecast
lgbm = LGBMRegressor(n_estimators=500)
LGBMforecaster = make_reduction(lgbm, window_length=12, strategy="recursive")
#Fits model on training data
LGBMforecaster.fit(y_train)
#Forecasts
lgbm_y_preds = LGBMforecaster.predict(fh)
#Calculates MAE for LGBM
lgbmMAE = mean_absolute_error(y_test, lgbm_y_preds)
#Prints 
#Prints MAE score for model and predictions from model
print('LGBM Model - MAE is %.6f' % (lgbmMAE))
print('--------LGBM Forecast--------')
lgbm_y_preds

LGBM Model - MAE is 9.871528
--------LGBM Forecast--------


36    94.076389
37    94.076389
38    94.076389
39    94.076389
40    94.076389
41    94.076389
42    94.076389
43    94.076389
44    94.076389
45    94.076389
46    94.076389
47    94.076389
dtype: float64

In [664]:

#Build Theta Model and score MAE on sample Forecast
theta = ThetaForecaster(sp=12)  
#THETAforecaster = make_reduction(theta, window_length=12, strategy='recursive')
#Fits on training data
theta.fit(y)  
#Forecasts
Theta_y_preds = theta.predict(fh)
#Calculates MAE for Theta 
thetaMAE = mean_absolute_error(y_test, Theta_y_preds)
#Prints MAE score for model and predictions from model
print('Theta Model - MAE is %.6f' % (thetaMAE))
print('--------Theta Forecast--------')
Theta_y_preds


Theta Model - MAE is 3.122123
--------Theta Forecast--------


48    103.490247
49    103.379537
50    103.728106
51    104.580414
52    104.412162
53    104.653024
54    105.510799
55    106.873632
56    105.544606
57    105.235887
58    105.420555
59    106.520083
Name: SG_Y, dtype: float64

In [665]:
mae_scores = np.array([RFmae, DTmae, XGBmae, KNNmae, autoArimaMAE, ArimaMAE, SarimaxMAE, Proph_mae, ETSmae, ESmae, BATSmae, TBATSmae, NaiveMAE, lgbmMAE, thetaMAE])

In [666]:
#   EVALUATION 
#Currently evaluates score of 15 regression models 
print('Random Forest Model - MAE is %.6f' % (RFmae))
print('Decision Tree Model - MAE is %.6f' % (DTmae))
print('XGBOOST Model       - MAE is %.6f' % (XGBmae))
print('K-NN Model          - MAE is %.6f' % (KNNmae))
print('Auto ARIMA Model    - MAE is %.6f' % (autoArimaMAE))
print('ARIMA Model         - MAE is %.6f' % (ArimaMAE))
print('SARIMAX Model       - MAE is %.6f' % (SarimaxMAE))
print('PROPHET Model       - MAE is %.6f' % (Proph_mae))
print('AUTO ETS Model      - MAE is %.6f' % (ETSmae))
print('Exponential S Model - MAE is %.6f' % (ESmae))
print('BATS Model          - MAE is %.6f' % (BATSmae))
print('TBATS Model         - MAE is %.6f' % (TBATSmae))
print('NAIVE Model         - MAE is %.6f' % (NaiveMAE))
print('LGBM Model          - MAE is %.6f' % (lgbmMAE))
print('Theta Model          - MAE is %.6f' % (thetaMAE))

Random Forest Model - MAE is 2.606042
Decision Tree Model - MAE is 2.579861
XGBOOST Model       - MAE is 2.593700
K-NN Model          - MAE is 2.559028
Auto ARIMA Model    - MAE is 6.104247
ARIMA Model         - MAE is 7.521689
SARIMAX Model       - MAE is 9.939516
PROPHET Model       - MAE is 6.942886
AUTO ETS Model      - MAE is 2.600905
Exponential S Model - MAE is 5.073832
BATS Model          - MAE is 3.050711
TBATS Model         - MAE is 3.050711
NAIVE Model         - MAE is 6.996528
LGBM Model          - MAE is 9.871528
Theta Model          - MAE is 3.122123


In [667]:
#   Find Best Fit Model
best_fit_model = np.amin(mae_scores)
worst_fit_model = np.amax(mae_scores)
print('Best MAE Result: ',best_fit_model, '    Worst MAE Result: ',worst_fit_model)
#   For this data it shows K-NN as the model best fit and SARIMAX as the worst fit

Best MAE Result:  2.5590277858333317     Worst MAE Result:  9.93951591190452


Multivariate VAR SKTIME Model

In [668]:
#Imports
#Using SKTIME 'load_longley' data for multivariate model
from sktime.datasets import load_longley
from sktime.forecasting.var import VAR
from sktime.performance_metrics.forecasting import mean_absolute_error

In [669]:
#Load data
_, y = load_longley()

#Show '_' data
_

Period
1947    60323.0
1948    61122.0
1949    60171.0
1950    61187.0
1951    63221.0
1952    63639.0
1953    64989.0
1954    63761.0
1955    66019.0
1956    67857.0
1957    68169.0
1958    66513.0
1959    68655.0
1960    69564.0
1961    69331.0
1962    70551.0
Freq: A-DEC, Name: TOTEMP, dtype: float64

In [670]:
#Show data
y

,GNPDEFL,GNP,UNEMP,ARMED,POP
Period,,,,,
1947,83.0,234289.0,2356.0,1590.0,107608.0
1948,88.5,259426.0,2325.0,1456.0,108632.0
1949,88.2,258054.0,3682.0,1616.0,109773.0
1950,89.5,284599.0,3351.0,1650.0,110929.0
1951,96.2,328975.0,2099.0,3099.0,112075.0
1952,98.1,346999.0,1932.0,3594.0,113270.0
1953,99.0,365385.0,1870.0,3547.0,115094.0
1954,100.0,363112.0,3578.0,3350.0,116219.0
1955,101.2,397469.0,2904.0,3048.0,117388.0


In [671]:
#Drop Columns
y = y.drop(columns=["UNEMP", "ARMED", "POP"])

y

,GNPDEFL,GNP
Period,,
1947,83.0,234289.0
1948,88.5,259426.0
1949,88.2,258054.0
1950,89.5,284599.0
1951,96.2,328975.0
1952,98.1,346999.0
1953,99.0,365385.0
1954,100.0,363112.0
1955,101.2,397469.0


In [672]:
#Build multivariate Model
#Define VAR Model
forecaster = VAR()
#Fit VAR Model
forecaster.fit(y, fh=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
#Forecasts
var_y_preds = forecaster.predict()
#Calculates MAE for BATS Model
varMAE = mean_absolute_error(y, var_y_preds)
#Prints MAE score for model
print('VAR Forecaster Model - MAE is %.6f' % (varMAE))
print('--------VAR Model Forecast--------')


var_y_preds


VAR Forecaster Model - MAE is 186908.124322
--------VAR Model Forecast--------


,GNPDEFL,GNP
1963,121.688295,578514.398653
1964,124.353664,601873.015890
1965,126.847886,625411.588754
1966,129.348210,649172.142647
1967,131.870921,673160.464237
1968,134.417707,697379.071966
1969,136.988932,721830.206958
1970,139.584842,746516.104508
1971,142.205677,771439.018889
1972,144.851675,796601.225796
